In [107]:
import pandas as pd
import numpy as np
hourly_bear_data = pd.read_pickle('hourly_bear.pkl')
minute_bull_data = pd.read_pickle('minutely_bull.pkl')
hourly_bull_data = pd.read_pickle('hourly_bull.pkl')
minute_crab_data = pd.read_pickle('minutely_crab.pkl')

In [108]:
# Conventional: no neural networks, just a typical MACD strategy
# loop
initial_investment = 10000.0 # USD
order_size = 1 # Full asset, since MACD is a one-open one-close signal
#No trading fees (eToro, Robinhood, Shakepay, Binance Spot (BTCUSD))

In [109]:
def mdd(df):
  max_investment = 0
  max_drawdown = 0
  for index, row in df.iterrows():
    asset = row['position_value'] + row['available_investment']
    if asset > max_investment:
      max_investment = asset
    if (max_investment - asset) / max_investment > max_drawdown:
      max_drawdown = (max_investment - asset) / max_investment
    # print(row['high'])
    df.at[index, 'mdd'] = max_drawdown
  return max_investment, max_drawdown

def get_returns(df):
    asset_current = df['position_value'] + df['available_investment']
    asset_past = df['position_value'].shift(-1) + df['available_investment'].shift(-1)
    df['macd_return'] = ((asset_current - asset_past) / asset_past) #df['close'].pct_change(1)
    return df
  
#100-period sortino ratio
def roll_sortino(df):
  risk_free = 0 #0 percent
  return_negative = df.to_numpy()
  return_negative_std = return_negative[return_negative < 0].std()
  return (df.mean() - risk_free) / return_negative_std * np.sqrt(100)

def sortino(df, time_period): #time period: m for minute, h for hour
  df['sortino'] = df['macd_return'].rolling('100{0}'.format(time_period)).apply(roll_sortino)

def sortino_average(df):
  return df['sortino'].to_numpy()[100:].mean()

In [110]:
def generate_macd_trading(df):
  df['macd_action'] = np.where(\
    (df['macd_histo'] * df['macd_histo'].shift(-1) >= 0), 'hold', \
    np.where((df['macd_histo'] > df['macd_histo'].shift(-1)), 'sell', 'buy') \
  )

  df['macd_action'] = df['macd_action'].shift(1)
  df['available_investment'] = initial_investment
  df['position'] = 0.0
  df['position_value'] = 0.0
  return df

def evaluate_macd(df):
  position_latest = 'idle'
  current_investment = initial_investment
  for index, row in df.iterrows():
    # print(row['macd_action'])
    if position_latest == 'idle':
      if row['macd_action'] == 'buy':
        position_latest = 'hold'
        asset_btc = current_investment / row['close']
        asset_value = row['close'] * asset_btc
        df.at[index, 'available_investment'] = 0.0
        df.at[index, 'position'] = asset_btc
        df.at[index, 'position_value'] = asset_value
        df.at[index, 'action_full'] = f'buy {asset_btc}'
        current_investment = asset_value
      elif row['macd_action'] == 'hold':
        df.at[index, 'available_investment'] = current_investment
    elif position_latest == 'hold':
      if row['macd_action'] == 'sell':
        asset_value = row['close'] * asset_btc
        df.at[index, 'position_value'] = asset_value
        df.at[index, 'position'] = asset_btc
        position_latest = 'idle'
        df.at[index, 'available_investment'] = asset_value
        df.at[index, 'action_full'] = f'sell {asset_btc}'
        current_investment = asset_value
        asset_btc = 0.0
        asset_value = 0.0
        df.at[index, 'position'] = asset_btc
        df.at[index, 'position_value'] = asset_value
      elif row['macd_action'] == 'hold':
        asset_value = row['close'] * asset_btc
        df.at[index, 'position_value'] = asset_value
        df.at[index, 'position'] = asset_btc
        df.at[index, 'available_investment'] = 0.0
        current_investment = row['close'] * row['position']
  return get_returns(df)

In [111]:
macd_tr_hbr = generate_macd_trading(hourly_bear_data)
macd_tr_hbu = generate_macd_trading(hourly_bull_data)
macd_tr_mcr = generate_macd_trading(minute_crab_data)
macd_tr_mbu = generate_macd_trading(minute_bull_data)

pd.options.display.max_rows = 500
macd_hourly_bear_data = evaluate_macd(generate_macd_trading(hourly_bear_data))
macd_hourly_bull_data = evaluate_macd(generate_macd_trading(hourly_bull_data))
macd_minute_crab_data = evaluate_macd(generate_macd_trading(minute_crab_data))
macd_minute_bull_data = evaluate_macd(generate_macd_trading(minute_bull_data))

In [112]:
macd_returns = {
  'name': 'macd_returns',
  'hourly_bear': (float(macd_hourly_bear_data.tail(1)['position_value']) + float(macd_hourly_bear_data.tail(1)['available_investment'])) / initial_investment - 1,
  'hourly_bull': (float(macd_hourly_bull_data.tail(1)['position_value']) + float(macd_hourly_bull_data.tail(1)['available_investment'])) / initial_investment - 1,
  'minutely_crab': (float(macd_minute_crab_data.tail(1)['position_value']) + float(macd_minute_crab_data.tail(1)['available_investment'])) / initial_investment - 1,
  'minutely_bull': (float(macd_minute_bull_data.tail(1)['position_value']) + float(macd_minute_bull_data.tail(1)['available_investment'])) / initial_investment - 1,
}

In [113]:
macd_returns

{'name': 'macd_returns',
 'hourly_bear': -0.3388918866897962,
 'hourly_bull': 0.21610747563212285,
 'minutely_crab': -0.014825836674962956,
 'minutely_bull': 0.03646763383285423}

In [114]:
macd_max_drawdown = {
  'name': 'macd_max_drawdown',
  'hourly_bear': mdd(macd_hourly_bear_data)[1],
  'hourly_bull': mdd(macd_hourly_bull_data)[1],
  'minutely_crab': mdd(macd_minute_crab_data)[1],
  'minutely_bull': mdd(macd_minute_bull_data)[1]
}

In [115]:
macd_max_drawdown

{'name': 'macd_max_drawdown',
 'hourly_bear': 0.35702332918743,
 'hourly_bull': 0.1279435136806022,
 'minutely_crab': 0.042708669556497415,
 'minutely_bull': 0.012905101082733536}

In [116]:
for i in [(macd_hourly_bear_data, 'h'), (macd_hourly_bull_data, 'h'), (macd_minute_crab_data, 'T'), (macd_minute_bull_data, 'T')]:
  sortino(i[0], i[1])

macd_sortino_ratio = {
  'name': 'macd_sortino_ratio',
  'hourly_bear': sortino_average(macd_hourly_bear_data),
  'hourly_bull': sortino_average(macd_hourly_bull_data),
  'minutely_crab': sortino_average(macd_minute_crab_data),
  'minutely_bull': sortino_average(macd_minute_bull_data)
}

c:\users\clementich\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice



In [117]:
macd_sortino_ratio

{'name': 'macd_sortino_ratio',
 'hourly_bear': 0.6350772595177102,
 'hourly_bull': 0.04209994904813594,
 'minutely_crab': 0.4893968346091791,
 'minutely_bull': -0.15922013269826624}

In [118]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [119]:
def plot_fig(df, title, custom_range=[0,1]):
  # Price line
  fig = make_subplots(rows=2, cols=1, 
                      specs = [[{"secondary_y": True}], 
                              [{"secondary_y": True}]])
  fig.update_layout(
      autosize=False,
      width=1300,
      height=800,
      title_text=title,
      yaxis_range=custom_range,
      yaxis2_range=[custom_range[0] - 1, custom_range[1] - 1]
    )

  initial_btc_price = float(df.head(1)['close'])
  fig.append_trace(
      go.Scatter(
          x=df.index,
          y=df['close'] / initial_btc_price,
          # line=dict(color='#ff9900', width=1),
          name='BTC/benchmark',
          # showlegend=False,
          legendgroup='1',
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='black',
        )
      ), row=1, col=1
  )

  # MACD Histogram
  colors = np.where(df['macd_histo'] < 0, '#000', '#ff9900')
  fig.append_trace(
      go.Bar(
          x=df.index,
          y=df['macd_histo'],
          name='Histogram',
          marker_color=colors,
      ), row=2, col=1
  )

  # Sortino
  fig.add_trace(
      go.Scatter(
          x=df.index,
          y=df['sortino'],
          name='Sortino',
      ), secondary_y=True, row=2, col=1
  )

  # Portfolio
  fig.add_trace(
    go.Scatter(
          x=df.index,
          y=(df['position_value'] + df['available_investment']) / initial_investment,
          # line=dict(color='#ff9900', width=1),
          name='Portfolio',
          # showlegend=False,
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='red',
        )
      ), row=1, col=1
  )

  # MDD
  fig.add_trace(
    go.Scatter(
          x=df.index,
          y=-df['mdd'],
          # line=dict(color='#ff9900', width=1),
          name='MDD',
          # showlegend=False,
          marker=dict(
          size=42,
          # I want the color to be green if 
          # lower_limit ≤ y ≤ upper_limit
          # else red
          color='blue',
        )
      ), secondary_y=True, row=1, col=1
  )
  return fig

In [120]:
fig_hr_bear = plot_fig(macd_hourly_bear_data, "MACD Hourly Bear Data", [0.45,1.05])
fig_hr_bull = plot_fig(macd_hourly_bull_data, "MACD Hourly Bull Data", [0.8,1.6])
fig_mi_crab = plot_fig(macd_minute_crab_data, "MACD Minute Crab Data", [0.95,1.05])
fig_mi_bull = plot_fig(macd_minute_bull_data, "MACD Minute Bull Data", [0.95,1.05])

In [121]:
def average_delta(data):
    init_price = float(data.head(1)['close'])
    init_inves = float(data.head(1)['available_investment'])
    comparison = ((data['available_investment'] + data['position_value']) / init_inves - data['close'] / init_price).dropna()
    return comparison.mean()

average_delta(macd_minute_bull_data)

-0.0007912511362626927

In [122]:
fig_hr_bear

In [123]:
fig_hr_bull

In [124]:
fig_mi_crab 

In [125]:
fig_mi_bull

In [126]:
def calculate_num_trades(df):
    trades = {
        'hold': 0,
        'buy': 0,
        'sell': 0
    }
    trade_value = {
        'buy': 0,
        'sell': 0
    }
    prev_action, value, last_row = 'hold', 0, None
    for index, row in df.iterrows():
        try:
          x = row['action_full'].split()
        except:
          x = ['hold', 0]
        action = x[0]
        action_val = float(x[1])
        try:
          trades[action] += 1
        except:
          pass
        if action == 'buy' or action == 'sell':
          # print(action, action_val)
          trade_value[action] += action_val
        prev_action, value, last_row = action, action_val, row
    
    if last_row['position'] > 0.0001:
      if last_row['macd_action'] == 'buy':
        # print(1)
        trades['buy'] - 1
        trades['sell'] += 1
        trade_value['buy'] -= last_row['position']
        trade_value['sell'] += last_row['position']
      elif last_row['macd_action'] == 'sell':
        # print(2)
        pass
      else:
        # print(3)
        trades['sell'] += 1
        trades['hold'] -= 1
        trade_value['sell'] += last_row['position']

    trade_avg = {
      'buy': trade_value['buy'] / trades['buy'],
      'sell': trade_value['sell'] / trades['sell'],
    }

    return trades, trade_value, trade_avg

print(calculate_num_trades(macd_tr_hbu))
print(calculate_num_trades(macd_tr_hbr))
print(calculate_num_trades(macd_tr_mcr))
print(calculate_num_trades(macd_tr_mbu))

({'hold': 1863, 'buy': 69, 'sell': 69}, {'buy': 40.06502231638123, 'sell': 40.06502231638123}, {'buy': 0.5806524973388584, 'sell': 0.5806524973388584})
({'hold': 1841, 'buy': 80, 'sell': 80}, {'buy': 14.083608837481414, 'sell': 14.083608837481414}, {'buy': 0.17604511046851767, 'sell': 0.17604511046851767})
({'hold': 1851, 'buy': 75, 'sell': 75}, {'buy': 30.747810487309366, 'sell': 30.747810487309366}, {'buy': 0.4099708064974582, 'sell': 0.4099708064974582})
({'hold': 1853, 'buy': 74, 'sell': 74}, {'buy': 34.00521997716701, 'sell': 34.00521997716701}, {'buy': 0.4595299996914461, 'sell': 0.4595299996914461})


In [127]:
macd_tr_hbr

,high,low,open,close,ewm,macd_histo,return,macd_action,available_investment,position,position_value,action_full,macd_return,mdd,sortino
time,,,,,,,,,,,,,,,
2021-11-09 06:00:00,68206.26,67740.93,68139.38,68054.30,68054.300000,0.000000,NaN,NaN,10000.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN
2021-11-09 07:00:00,68162.86,67696.73,68054.30,67898.05,68043.524138,-9.971510,-0.002296,hold,10000.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN
2021-11-09 08:00:00,68143.44,67743.56,67898.05,68116.79,68048.576956,-1.716533,0.003222,hold,10000.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN
2021-11-09 09:00:00,68241.40,67909.08,68116.79,68240.99,68061.846821,11.500303,0.001823,buy,0.000000,0.146539,10000.000000,buy 0.14653949188017348,0.005492,0.000000,NaN
2021-11-09 10:00:00,68249.47,67850.14,68240.99,67868.27,68048.496696,-4.604197,-0.005462,sell,9945.381801,0.000000,0.000000,sell 0.14653949188017348,0.000000,0.005462,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 10:00:00,37357.00,37175.50,37231.37,37316.60,37441.917076,9.403060,0.002289,buy,0.000000,0.176122,6572.256902,buy 0.176121535771122,0.000637,0.353270,1.287766
2022-01-31 11:00:00,37355.52,36913.55,37316.60,37292.85,37431.636588,22.771743,-0.000636,hold,0.000000,0.176122,6568.074015,NaN,0.005197,0.353682,1.414273
2022-01-31 12:00:00,37419.24,36867.46,37292.85,37100.04,37408.767858,19.433723,-0.005170,hold,0.000000,0.176122,6534.116022,NaN,-0.005011,0.357023,1.323313
